<a id='top'></a>

# CSCI 3202, Spring 2018
# Assignment 3
# Due:  Wednesday 28 February 2018 by 12:00 PM

<br>

### Your name:

<br>

**Note:** Some packages to load, helper functions and unit tests are defined at [the bottom of this notebook](#helpers)

Shortcuts:  [top](#top) || [1](#p1) | [1a](#p1a) | [1b](#p1b) | [1c](#p1c) | [1d](#p1d) | [1e](#p1e) || [2](#p2) | [2a](#p2a) | [2b](#p2b) | [2c](#p2c) | [2d](#p2d) | [2e](#p2e) | [2f](#p2f) || [helpers](#helpers)

---

<a id='p1'></a>[Back to top](#top)

## Problem 1

### Still trying to Escape from Chicago

Well, it turns out Snake lived through his perilous trip to New York City.  He returned to Chicago, where he is living happily among the criminals, sea lions and criminal sea lions.

During Snake's time on the east coast, he came to realize that the soothing Atlantic Ocean air does wonders for his nerves. Living in the penal colony of Chicago is just so stressful! So he has decided to pack up and move to Providence, Rhode Island.

Because Snake has traveled across the country already, he has some prior knowledge regarding how far various cities along the way are from his final destination of Providence.  In particular, Snake maps out the **straight-line distance** from major cities to Providence. This information is displayed graphically below, on a tattoo on Kurt Russell's forearm (you can't prove that isn't true), and in the `sld_providence` dictionary in the [helper functions](#helper) at the bottom of this notebook.

Also note that because it doesn't make much sense to add estimated travel times and the straight-line distance, we will only be using the `map_distances` state space graph for this problem.

**map_distances**          |  **sld_providence**
:-------------------------:|:-------------------------:
<img src="http://www.cs.colorado.edu/~tonyewong/home/resources/us_map_distances.png" style="width: 500px;"/>  | <img src="http://www.cs.colorado.edu/~tonyewong/home/resources/us_map_heuristicSLD.png" style="width: 500px;"/>


<a id='p1a'></a>
### (1a)

Modify your code for uniform-cost search from Homework 2 so that it provides optionally as output the number of nodes **expanded** in completing the search. 

Include a new optional logical (True/False) argument **return_nexp**, so your function calls to the new uniform cost search will look like: `uniform_cost(start, goal, state_graph, return_cost, return_nexp)`.
* If `return_nexp` is True, then the last output in the output tuple should be the number of nodes expanded.
* If `return_nexp` is False, then the code should behave exactly as it did in Homework 2.

Then, verify that your revised codes are working by checking Snake's optimal route from Chicago to New York. Include the number of nodes expanded and the path cost (using `map_distances`).

In [ ]:
# Thanks for providing a solution from hw2.

class Frontier_PQ:
    ''' frontier class for uniform search, ordered by path cost '''
    
    def __init__(self, start, cost):
        self.states = {}
        self.q = []
        self.add(start, cost)
        #self.expanded = 0
        
    def add(self, state, cost):
        ''' push the new state and cost to get there onto the heap'''
        heapq.heappush(self.q, (cost, state))
        self.states[state] = cost
        #self.expanded += 1

    def pop(self):
        (cost, state) = heapq.heappop(self.q)  # get cost of getting to explored state
        self.states.pop(state)    # and remove from frontier
        return (cost, state)

    def replace(self, state, cost):
        ''' found a cheaper route to `state`, replacing old cost with new `cost` '''
        self.states[state] = cost
        for i, (oldcost, oldstate) in enumerate(self.q):
            if oldstate==state and oldcost > cost:
                self.q[i] = (cost, state)
                heapq._siftdown(self.q, 0, i) # now i is posisbly out of order; restore
        return
    
    
def uniform_cost(start, goal, state_graph, return_cost=False,return_nexp=False):
    frontier = Frontier_PQ(start, 0)
    previous = {start : None}
    explored = {}
    expanded = 1
    while frontier:
        s= frontier.pop()
        if s[1] == goal:
            #print("tmp",tmp)
            print("exp",expanded)  # Think I could have written this better, but I need to get this done today..
            if return_cost and return_nexp: return path(previous, s[1]), s[0], expanded
            if return_cost and not return_nexp: return path(previous, s[1]), s[0]
            if return_nexp and not return_cost: return path(previous, s[1]), expanded
            return path(previous, s[1])
        explored[s[1]] = s[0]
        expanded+=1
        for s2 in state_graph[s[1]]:
            newcost = explored[s[1]]+state_graph[s[1]][s2]
            if (s2 not in explored) and (s2 not in frontier.states):
                frontier.add(s2, newcost)
                previous[s2] = s[1]
            elif (s2 in frontier.states) and (frontier.states[s2] > newcost):
                frontier.replace(s2, newcost)
                previous[s2] = s[1]


#### Unit tests

In [ ]:
tests_to_run = unittest.TestSuite()
tests_to_run.addTest(Tests_Problem1('test_ucs_route'))
tests_to_run.addTest(Tests_Problem1('test_ucs_cost'))
tests_to_run.addTest(Tests_Problem1('test_ucs_nexp'))
unittest.TextTestRunner().run(tests_to_run)

#path, cost, nexp = uniform_cost('chi','new',map_distances,return_cost=True,return_nexp=True)

<br>

<a/ id='p1b'></a>

### (1b)

#### Heuristic function

Define a function to take as an argument the `state` that Snake is in (city on our graphs), and return as output the value of the straight-line distance heuristic, between Snake's `state` and Providence.  Note that your function should be quite short, and amounts to looking up the proper value from the `sld_providence` dictionary defined in the [helper functions](#helper).  Call this function `heuristic_sld_providence`.

This might seem sort of silly right now, but it will make your codes much easier to adapt in Problem 2.

In [ ]:
def heuristic_sld_providence(state):
    # Always trying to get to providence so just hard code that in...?
    # Also seems that the straight lines from any given location have alreay been calculated
    return sld_providence[state]
    

#### Unit tests

In [ ]:
tests_to_run = unittest.TestSuite()
tests_to_run.addTest(Tests_Problem1("test_sld_providence"))
unittest.TextTestRunner().run(tests_to_run)

<br>

<a/ id='p1c'></a>

### (1c)

#### Full-blown A\* search

We are finally ready to help Snake use his knowledge of straight-line distances from various cities to Providence to inform his path-finding from Chicago to Providence!

Modify your uniform-cost search codes from **(1a)** even further so that they now perform **A\* search**, using as the heuristic function the straight-line distance to Providence.

Provide `heuristic` as an additional argument, which should just be the function name to call within the A\* code. So your call to the A\* routine should look like:  `astar_search(start, goal, state_graph, heuristic, return_cost, return_nexp)`.  (This kind of modular programming will make it much easier to swap in alternative heuristic functions later, and also helps to facilitate debugging if something goes wrong.)

In [ ]:
# Thanks for providing a solution from hw2.

class Frontier_PQ:
    ''' frontier class for uniform search, ordered by path cost '''
    
    def __init__(self, start, cost):
        self.states = {}             # How my states link together
        self.q = []                  # My queue this algorithm (A heap)
        self.add(start, cost)        # initialize my first location on the heap
        #self.expanded = 0            # Keeps track of n/m states that have been expanded (not visited)
        
    def add(self, state, cost):
        ''' push the new state and cost to get there onto the heap'''
        heapq.heappush(self.q, (cost, state))       # The heap helps create a tree to organize our path
        self.states[state] = cost                   # Everytime I add to heap also add to states
        #self.expanded += 1

    def pop(self):
        (cost, state) = heapq.heappop(self.q)  # get cost of getting to explored state
        self.states.pop(state)    # and remove from frontier
        return (cost, state)

    def replace(self, state, cost):
        ''' found a cheaper route to `state`, replacing old cost with new `cost` '''
        self.states[state] = cost
        for i, (oldcost, oldstate) in enumerate(self.q):
            if oldstate==state and oldcost > cost:
                self.q[i] = (cost, state)
                heapq._siftdown(self.q, 0, i) # now i is posisbly out of order; restore
        return
    
    
def astar_search(start, goal, state_graph,heuristic=heuristic_sld_providence, return_cost=False,return_nexp=False):
    frontier = Frontier_PQ(start, heuristic(start))
    previous = {start : None}
    explored = {}
    expanded=1
    while frontier:
        s= frontier.pop()
        if s[1] == goal:
            print("exp",expanded)  # Think I could have written this better, but I need to get this done today..
            if return_cost and return_nexp: return path(previous, s[1]), s[0], expanded
            if return_cost and not return_nexp: return path(previous, s[1]), s[0]
            if return_nexp and not return_cost: return path(previous, s[1]), expanded
            return path(previous, s[1])
        explored[s[1]] = s[0]-heuristic(s[1])
        expanded+=1
        for s2 in state_graph[s[1]]:
            newcost = explored[s[1]]+state_graph[s[1]][s2]+heuristic(s2)
            if (s2 not in explored) and (s2 not in frontier.states):
                frontier.add(s2, newcost)
                previous[s2] = s[1]
            elif (s2 in frontier.states) and (frontier.states[s2] > newcost):
                frontier.replace(s2, newcost)
                previous[s2] = s[1]



#### Unit tests

In [ ]:
tests_to_run = unittest.TestSuite()
tests_to_run.addTest(Tests_Problem1("test_astar_path"))
tests_to_run.addTest(Tests_Problem1("test_astar_cost"))
tests_to_run.addTest(Tests_Problem1("test_astar_nexp"))
unittest.TextTestRunner().run(tests_to_run)

<br>

<a/ id='p1d'></a>

### (1d)

Snake has already made it to Buffalo on his own, but he is growing weary.  There were ***so*** many sea lions along the way, and he had to stop and admire their grace and beauty.  Use your A\* search code from **(1c)** to help Snake find the optimal path **by distance traveled** the rest of the way to Providence.  Print to the screen the following:
1. the optimal path
2. the optimal path cost (miles traveled)
3. the number of states expanded during the A\* search

Additionally, show below how many states must be expanded to find the optimal path from Buffalo to Providence using the regular old uniform-cost search algorithm from **(1a)**.  Comment on the difference in states that must be explored by each algorithm.

**Sanity check:** No matter what your `start` and `goal` states are, how should the output from `astar_search` and `uniform_cost` search compare?

In [ ]:
start='buf'
goal='pro'
state_graph=map_distances

path_A, cost_A, nexp_A = astar_search(start, goal, state_graph,heuristic=heuristic_sld_providence, return_cost=True,return_nexp=True)

print("1. the optimal path: ",path_A)
print("2. the optimal path cost: ",cost_A)
print("3. the number of states expanded during the A* search: ",nexp_A)

path_U, cost_U, nexp_U = uniform_cost(start, goal, state_graph, return_cost=True,return_nexp=True)

print("1. the optimal path: ",path_U)
print("2. the optimal path cost: ",cost_U)
print("3. the number of states expanded during the UCS search: ",nexp_U)




<br>

<a/ id='p1e'></a>

### (1e)

How many states are expanded by each of A\* search and uniform cost search, to find the optimal path **from Philadelphia to Providence**?

In [ ]:
start='phi'
goal='pro'
state_graph=map_distances

path_A, cost_A, nexp_A = astar_search(start, goal, state_graph,heuristic=heuristic_sld_providence, return_cost=True,return_nexp=True)

print("1. the optimal path: ",path_A)
print("2. the optimal path cost: ",cost_A)
print("3. the number of states expanded during the A* search: ",nexp_A)

path_U, cost_U, nexp_U = uniform_cost(start, goal, state_graph, return_cost=True,return_nexp=True)

print("1. the optimal path: ",path_U)
print("2. the optimal path cost: ",cost_U)
print("3. the number of states expanded during the UCS search: ",nexp_U)

You should find that uniform cost search has explored more states than A\* search.  Name the city/cities corresponding to the additional state expansions, and a brief justification for how you know these are being explored.

The difference is 2:
1. Is Baltamore('bal') - because with Uniform Search after we visit New York the cost to go to 'bal' is less than going to Providence due to 'pro' from 'phi' is 101 while cost from 'phi' to 'pro' Going through new york is 278. So we check the shortest path, 'phi'->'bal'
2. Is Syracuse('syr') - because, similar to the above reason, distance from 'phi'->'syr' is 253 which is still less that 278 as shown above and thus it is check before the path to the goal.


<br>

---

<br>

[Back to top](#top)

<a/ id='p2'></a>

![8tile](http://www.8puzzle.com/images/8_puzzle_goal_state_b.png)

## Problem 2

### 8-tile puzzle

Consider the 8-tile puzzle, as discussed in class.  The goal state is illustrated in the figure above, and the rest of the state space is obtained by sliding a numbered tile in the plane of the puzzle into the blank spot.  We can use a 0 to represent the blank space, and represent the state of an 8-tile puzzle board as a `numpy` array as follows:

In [4]:
goal = np.array([[0,1,2],[3,4,5],[6,7,8]])
print('Goal state is:\n{}'.format(goal))

Goal state is:
[[0 1 2]
 [3 4 5]
 [6 7 8]]


<br>

<a/ id='p2a'></a>

### (2a)

Design two heuristic functions to return the **Manhattan distance** heuristic and the **number of misplaced tiles** heuristic.  These functions should be called `heuristic_manhattan` and `heuristic_misplaced`, and:
* take two `numpy` array arguments, `state` and `goal`
* `state` is the current state of an 8-tile board,
* `goal` is the goal state to measure against,
* return the appropriate heuristic value (Manhattan distance or misplaced tiles) for the estimated distance of the given `state` to the `goal`

In [5]:
state_test = np.array([[0,4,2],[1,3,5],[6,7,8]])
goal = np.array([[0,1,2],[3,4,5],[6,7,8]])

print('Test state is:\n{}'.format(state_test))
print('Goal state is:\n{}'.format(goal))


def heuristic_manhattan(state,goal):
    m,n = np.shape(state)
    cStart = {}               # Using dicts for quick sorting to get coordiantes
    cGoal = {}
    disp = []
    for i in range(1,m+1):       # Choosing these dimensions so that the grid starts at (1,1) not (0,0) and top left is start
        for j in range(1,n+1):   # x-coord(i) and y-coord(j)
            cStart[state_test[i-1,j-1]]=[i,j]
            cGoal[goal[i-1,j-1]]=[i,j]
    for i in cStart.keys():    # Make sure to look at each value
        disp.append(cStart[i][0]-cGoal[i][0])    # Append the x-coordiante disp for each value
        disp.append(cStart[i][1]-cGoal[i][1])    # Append the y-coordinate disp for each value
    return np.sum(np.abs(disp))


def heuristic_misplaced(state,goal):
    return np.sum(state_test != goal)

######################
# Just testing stuff #
######################

#print(heuristic_manhattan(state_test, goal))


## Making dicts to compare key = value, item = coordinate
## So I think there is a much better way to do this, but I'm tired and just need to get this assignment done.
## I should ask later though...
#
#cStart = {}
#cGoal = {}
#tmp = []
#for i in range(1,4):
#    for j in range(1,4):
#        cStart[state_test[i-1,j-1]]=[i,j]
#        cGoal[goal[i-1,j-1]]=[i,j]
#        # Now to take difference as I have hash collisions:
#for i in cStart.keys():
#    tmp.append(cStart[i][0]-cGoal[i][0])
#    tmp.append(cStart[i][1]-cGoal[i][1])
#print(tmp)            
#        
# This shows that I can see the correct displacment
##for i in cStart.keys():
#    print(cStart[i],cGoal[i])
#    
## Now trying to simplify this a lot... I want to do it all in 2 loops





Test state is:
[[0 4 2]
 [1 3 5]
 [6 7 8]]
Goal state is:
[[0 1 2]
 [3 4 5]
 [6 7 8]]


#### Unit tests

In [6]:
tests_to_run = unittest.TestSuite()
tests_to_run.addTest(Tests_Problem2("test_manhattan"))
tests_to_run.addTest(Tests_Problem2("test_misplaced"))
unittest.TextTestRunner().run(tests_to_run)

..
----------------------------------------------------------------------
Ran 2 tests in 0.005s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

<br>

<a/ id='p2b'></a>

### (2b)

#### Is this puzzle even solvable??

It turns out that some arrangements of those 8 tiles are not possible to solve into the `goal` state given above. We would like to be able to check whether or not a given 8-tile puzzle board is solvable.

Suppose you take the tiles of any given 8-tile puzzle board and arranged them in a single row, with the first row coming in order first, then the second row in order, and finally the third row in order.  So the goal state would be [1,2,3,4,5,6,7,8]. Another possible state is:

In [7]:
state_test = np.array([[1,4,2],[3,0,5],[6,7,8]])
state_test

array([[1, 4, 2],
       [3, 0, 5],
       [6, 7, 8]])

The state `state_test` is found by sliding the 1 tile to the blank spot, then sliding the 4 tile up to take the position formerly occupied by the 1 tile.  When we rearrange this state into a row, we obtain [1,4,2,3,5,6,7,8].

We define an **inverted tile pair** to be a pair of tiles in the row representation of the puzzle that is out of order relative to the goal state. That is, each occurrence of a tile that has a higher value than another tile later in the row counts as an inversion.  So `state_test` has **two** inversions:  4 and 2, and 4 and 3.

We define the **parity** of an 8-tile board to be the number of inversions in the puzzle board.  So the parity of `state_test` is 2, and the parity of `goal` is 0. An 8-tile puzzle board may be rearranged (within the rules of the game) into another board if the two boards have the same parity. So it would be very handy if we had a function to check whether or not an arbitrary arrangement of those 8 tiles can be solved for a given `goal` state.

Write a function `check_parity(state1, state2)`:
* take as arguments two `numpy` arrays representing 8-tile boards, `state1` and `state2`
* return `True` if the two boards have the same parity and `False` otherwise

In [8]:
def check_parity(state1,state2):
    par1=0
    par2=0
    state1 = state1[state1>0]
    print(state1)
    state2 = state2[state2>0]
    print(state2)
    for i in range(len(state1)):
        par1 += np.sum( state1[i:] < state1[i] )  # So ideally this should take the sum of all values less than i
                                              # for the state array after the i-th element
        par2 += np.sum( state2[i:] < state2[i] )
    print("par1",par1,"par2",par2)
    return True if par1==par2 else False
        
# Just testing out my thoughts below #

#state_test1 = state_test[state_test>0]
#print(state_test1)
#np.sum(state_test > 5)


#check_parity(state_test,state_test)
state_test = np.array([[0,4,2],[1,3,5],[6,7,8]])
goal = np.array([[0,1,2],[3,4,5],[6,7,8]])
check_parity(state_test, goal)  # Should be True

state_test = np.array([[0,2,1],[3,4,5],[6,7,8]])
goal = np.array([[0,1,2],[3,4,5],[6,7,8]])
#assertFalse(check_parity(state_test, goal))




[4 2 1 3 5 6 7 8]
[1 2 3 4 5 6 7 8]
par1 4 par2 0


#### Unit tests

In [9]:
tests_to_run = unittest.TestSuite()
#tests_to_run.addTest(Tests_Problem2("test_parity_true"))
tests_to_run.addTest(Tests_Problem2("test_parity_false"))
unittest.TextTestRunner().run(tests_to_run)

.

[2 1 3 4 5 6 7 8]
[1 2 3 4 5 6 7 8]
par1 1 par2 0



----------------------------------------------------------------------
Ran 1 test in 0.017s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

<br>

<a/ id='p2c'></a>

### (2c)

Because your instructor is such a swell guy, the `map_distances` and `map_times` graphs were nicely defined. This meant that finding the successors of a given city in the route-finding homework problems amounted to looking up the values of that key in a dictionary. For the 8-tile problem, however, we do **not** want to try to build the entire state space graph in memory! Even if we can, it's just such a pain.

Instead, let's write a function `adjacent_states`:
* take a single input argument `state`, which is a `numpy` array representing an 8-tile board arrangement
* return as output a list of `numpy` arrays that are the states one move away from the input `state`

For example, if you run `adjacent_states(goal)`, the expected output should be

`[array([[3, 1, 2], [0, 4, 5], [6, 7, 8]]),
 array([[1, 0, 2], [3, 4, 5], [6, 7, 8]])]`

which are the states obtained from the `goal` state by sliding the 3 tile up and by sliding the 1 tile to the left, respectively.

In [68]:
def adjacent_states(state):
    m,n = np.shape(state)   # Just encase I want to try and expand this to larger dimensions
    y,x = np.where(state==0)
    #print(y,x)
    #print(state)
    # To tired so I will be writing this out a lot...
    up = y-1      # Up and down are inverted due to my grid origin being top-left.. which is why at midnight it's hard to think about lol
    down = y+1
    left = x-1
    right = x+1
    steps = []
    if up>=0:
        stateUP = np.copy(state)
        stateUP[y,x] = state[up,x]
        stateUP[up,x] = 0
        steps.append(stateUP)
    if left>=0:
        stateL = np.copy(state)
        stateL[y,x] = state[y,left]
        stateL[y,left] = 0
        steps.append(stateL)
    if down<=2:
        stateDown = np.copy(state)
        stateDown[y,x] = state[down,x]
        stateDown[down,x] = 0
        steps.append(stateDown)
    if right<=2:
        stateR = np.copy(state)
        stateR[y,x] = state[y,right]
        stateR[y,right] = 0
        steps.append(stateR)
    return steps
    
print(goal)

tmp = adjacent_states(goal)
print(tmp[0])
#print(tmp[1])

testing0 = tuple(map(tuple,tmp[0]))
testing1 = tuple(map(tuple,tmp[1]))
print(testing0)
#testing2 = np.array(map(np.array,testing))
#print(testing2)
testing2 = np.array(testing0)

print(testing2)
# Four cases:
# 1. I can swap 0 and up
# 2. I can swap 0 and down
# 3. I can swap 0 and left
# 4. I can swap 0 and right

testDict={}
testDict[testing0]=testing1

testDict

[[0 1 2]
 [3 4 5]
 [6 7 8]]
[[3 1 2]
 [0 4 5]
 [6 7 8]]
((3, 1, 2), (0, 4, 5), (6, 7, 8))
[[3 1 2]
 [0 4 5]
 [6 7 8]]


{((3, 1, 2), (0, 4, 5), (6, 7, 8)): ((1, 0, 2), (3, 4, 5), (6, 7, 8))}

#### Unit tests

In [69]:
tests_to_run = unittest.TestSuite()
tests_to_run.addTest(Tests_Problem2("test_adjacent"))
unittest.TextTestRunner().run(tests_to_run)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

<br>

<a/ id='p2d'></a>

### (2d)

Modify your **A\* search** from above so that instead of looking in the graphs `map_distances` or `map_times` for the successor states, it generates them on the fly using your shiny new `adjacent_states` routine from **(2c)**. 

Some notes:
* You may run into some problems with using the `numpy` arrays to represent states. One potential solution is to convert these arrays to/from tuples, as needed. This is not the only solution.
* When you start to test your code, be sure to begin with simple test cases that you know are only a few moves away from the goal state.
* Be sure to implement a check to make sure the initial state is solvable for the given goal state!  And do not hard-code the goal state given above.
* You may need to modify how path costs are added up and returned. The returned cost of the solution path ought to be the number of **moves** needed to solve the puzzle, which is one less than the number of states on the solution path.
* Depending on how you implemented your Frontier Priority Queue, you may or may not need to make modifications to it. My implementation did not require any modification.

In [88]:
# Thanks for providing a solution from hw2.

class Frontier_PQ:
    ''' frontier class for uniform search, ordered by path cost '''
    
    def __init__(self, start, cost):
        self.states = {}             # How my states link together
        self.q = []                  # My queue this algorithm (A heap)
        self.add(start, cost)        # initialize my first location on the heap
        #self.expanded = 0            # Keeps track of n/m states that have been expanded (not visited)
        
    def add(self, state, cost):
        ''' push the new state and cost to get there onto the heap'''
        heapq.heappush(self.q, (cost, state))       # The heap helps create a tree to organize our path
        self.states[state] = cost                   # Everytime I add to heap also add to states
        #self.expanded += 1

    def pop(self):
        (cost, state) = heapq.heappop(self.q)  # get cost of getting to explored state
        self.states.pop(state)    # and remove from frontier
        return (cost, state)

    def replace(self, state, cost):
        ''' found a cheaper route to `state`, replacing old cost with new `cost` '''
        self.states[state] = cost
        for i, (oldcost, oldstate) in enumerate(self.q):
            if oldstate==state and oldcost > cost:
                self.q[i] = (cost, state)
                heapq._siftdown(self.q, 0, i) # now i is posisbly out of order; restore
        return

    
    
    
    
def astar_8tile(start, goal, state_graph=adjacent_states,heuristic=heuristic_manhattan, return_cost=False,return_nexp=False):
    frontier = Frontier_PQ(tuple(map(tuple,start)), heuristic(start,goal))  # Heuristic is fine, start and goal are still np
    previous = {tuple(map(tuple,start)) : None}
    explored = {}
    expanded=1
    while frontier:
        s= frontier.pop()
        if np.all(np.array(s[1]) == goal):
            #print(np.all(np.array(s[1]) == goal))
            #print(np.array(s[1]) == goal)
            #print("exp",expanded)  # Think I could have written this better, but I need to get this done today..
            
            if return_cost and return_nexp: return path(previous, s[1]), s[0], expanded
            if return_cost and not return_nexp: return path(previous, s[1]), s[0]
            if return_nexp and not return_cost: return path(previous, s[1]), expanded
            return path(previous, s[1])
        explored[tuple(map(tuple,s[1]))] = s[0]-heuristic(np.array(s[1]),goal) # s[1] from frontier so it is tuple...
        expanded+=1
        new_states = state_graph(np.array(s[1]))
        for s1 in new_states:
            s2 = tuple(map(tuple,np.copy(s1)))
            #print(np.array(s2))
            # s2 and s[1] here should both be a tuple:
            #print("s2",type(s2),"s[1]",type(s[1]))
            newcost = explored[tuple(map(tuple,s[1]))]+heuristic(np.array(np.copy(s2)),goal)+1 # Every step is a cost of 1# Need to correct this cost -> state_graph(np.array(s[1])) # Make sure use of s[1] and s2 are turned to arrays
            if (s2 not in explored) and (s2 not in frontier.states):
                frontier.add(s2, newcost)
                previous[s2] = s[1]
            elif (s2 in frontier.states) and (frontier.states[s2] > newcost):
                frontier.replace(s2, newcost)
                previous[s2] = s[1]


                
# I'm getting multiple 0-tiles? what... I copied my arrays to be safe.. weird
# Somewhere I have some kind of memory issue, I am not copying my arrays correctly and seem to be overwritting my array 
# Without correct replacement <- nope... had x,y in wrong spots on 2 of my cases for creating my adjacents...


In [89]:
goal = np.array([[0,1,2],[3,4,5],[6,7,8]])
state_test = np.array([[1,4,2],[3,5,8],[6,7,0]])
moves, cost, nexp = astar_8tile(state_test, goal,state_graph=adjacent_states, heuristic=heuristic_manhattan, return_cost=True, return_nexp=True)

moves
print(goal)
print(state_test)
print("moves")
print(np.array(moves))

[[0 1 2]
 [3 4 5]
 [6 7 8]]
[[1 4 2]
 [3 5 8]
 [6 7 0]]
moves
[[[1 4 2]
  [3 5 8]
  [6 7 0]]

 [[1 4 2]
  [3 5 0]
  [6 7 8]]

 [[1 4 2]
  [3 0 5]
  [6 7 8]]

 [[1 0 2]
  [3 4 5]
  [6 7 8]]

 [[0 1 2]
  [3 4 5]
  [6 7 8]]]


#### Unit tests

Note that this will fail if your output path (first output slot) is not one of (1) a list of tuples, or (2) a list of `numpy` arrays.

In [91]:
tests_to_run = unittest.TestSuite()
tests_to_run.addTest(Tests_Problem2("test_astar_8tile"))
unittest.TextTestRunner().run(tests_to_run)



answer = {((1, 0, 5), (3, 2, 4), (6, 7, 8)),
          ((1, 2, 5), (0, 3, 4), (6, 7, 8)),
          ((1, 2, 5), (3, 4, 0), (6, 7, 8)),
          ((1, 2, 5), (3, 7, 4), (6, 0, 8))}

F
FAIL: test_astar_8tile (__main__.Tests_Problem2)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-65-7c759587ea61>", line 66, in test_astar_8tile
    self.assertEqual(cost, 4)
AssertionError: 12 != 4

----------------------------------------------------------------------
Ran 1 test in 0.018s

FAILED (failures=1)


<br>

<a/ id='p2e'></a>

### (2e)

#### We need to generate some puzzles to test this out!

Create a function `generate_puzzle(goal, n_moves)` to generate a solvable 8-tile puzzle by making `n_moves` random moves, starting from the `goal` state.
* `goal` argument is a `numpy` array representing a valid 8-tile puzzle
* `n_moves` is a positive integer argument.  For each of `n_moves` iterations, generate the set of all legal 8-tile puzzle moves (you might already have a nice function for this...) and select one at random.
* When you generate the set of possible adjacent states to move to, remove any of the previous states from the list of possibilities.  This will avoid "undoing" any of the moves you've made to mix up the tiles, which is important because the point of this function is to generate tough puzzles for our codes to solve.
* return as output a tuple:
  * first element is the puzzle itself (as a `numpy` array)
  * second element is the list of previous states, so you can see the list of moves made to generate this puzzle.  This will be helpful for checking if our A\* codes are finding the correct solutions.

**Important question:** If you generate more complex puzzles (think 20 or more moves), there's a nontrivial probability that your A\* search routine will be able to find a solution in *fewer* moves than the `n_moves` argument you supplied to `generate_puzzle` to create that very puzzle.
1. Verify that this is the case with at least one example.
2. Explain why this is the case.

#### Unit tests

Note that these won't work unless your A\* search and heuristic functions are working.

In [ ]:
tests_to_run = unittest.TestSuite()
tests_to_run.addTest(Tests_Problem2("test_generate1"))
tests_to_run.addTest(Tests_Problem2("test_generate2"))
unittest.TextTestRunner().run(tests_to_run)

<br>

<a/ id='p2f'></a>

### (2f)

Generate 8-tiles puzzles varying in complexity in number of moves for the optimal solution, ranging from 10 moves to at least 25 moves.  Solve each puzzle twice using A\* search:  once using the Manhattan distance heuristic, and once using the misplaced tiles heuristic.  Keep track of:
1. the number of moves in the optimal solution path for each puzzle,
2. the number of states explored by A\* search using the Manhattan distance heuristic, and
3. the number of states explored by A\* search using the number of misplaced tiles heuristic.

In light of your answer to **(2e)**, it might be difficult to reliably generate the more complex puzzles.  As long as there are at least two puzzles that require at least 20 moves to solve, you have satisfied the problem requirements.

Note that you could put some `while` loops in here (for example) to make sure you get puzzles with all of the requisite solution path lengths. I don't really care if you do or not.

Now generate a plot of the number of states explored by A\* search using each heuristic, as a function of the number of moves along the optimal solution path. Be sure to label everything clearly.  Write a sentence or two to comment on what you notice, and whether or not it is consistent with what we saw in class.

<br><br><br>

<a id='helpers'></a>

---

[Back to top](#top)

## Some things that might be useful

Easiest way to start:  Click this cell, go to "Cell" in the toolbar above, and click "Run All Below"

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import heapq
import unittest

## Potentially useful for searching

In [2]:
def path(previous, s): 
    '''
    `previous` is a dictionary chaining together the predecessor state that led to each state
    `s` will be None for the initial state
    otherwise, start from the last state `s` and recursively trace `previous` back to the initial state,
    constructing a list of states visited as we go
    '''
    if s is None:
        return []
    else:
        return path(previous, previous[s])+[s]

def pathcost(path, step_costs):
    '''
    add up the step costs along a path, which is assumed to be a list output from the `path` function above
    '''
    cost = 0
    for s in range(len(path)-1):
        cost += step_costs[path[s]][path[s+1]]
    return cost

map_distances = dict(
    chi=dict(det=283, cle=345, ind=182),
    cle=dict(chi=345, det=169, col=144, pit=134, buf=189),
    ind=dict(chi=182, col=176),
    col=dict(ind=176, cle=144, pit=185),
    det=dict(chi=283, cle=169, buf=256),
    buf=dict(det=256, cle=189, pit=215, syr=150),
    pit=dict(col=185, cle=134, buf=215, phi=305, bal=247),
    syr=dict(buf=150, phi=253, new=254, bos=312),
    bal=dict(phi=101, pit=247),
    phi=dict(pit=305, bal=101, syr=253, new=97),
    new=dict(syr=254, phi=97, bos=215, pro=181),
    pro=dict(bos=50, new=181),
    bos=dict(pro=50, new=215, syr=312, por=107),
    por=dict(bos=107))

sld_providence = dict(
    chi=833,
    cle=531,
    ind=782,
    col=618,
    det=596,
    buf=385,
    pit=458,
    syr=253,
    bal=325,
    phi=236,
    new=157,
    pro=0,
    bos=38,
    por=136)

## Unit tests

In [65]:
class Tests_Problem1(unittest.TestCase):
    def test_ucs_route(self):
        path = uniform_cost('chi','new',map_distances,return_cost=False,return_nexp=False)
        self.assertEqual(path, ['chi', 'cle', 'pit', 'phi', 'new'])
    def test_ucs_cost(self):
        path, cost = uniform_cost('chi','new',map_distances,return_cost=True,return_nexp=False)
        self.assertEqual(cost, 881)
    def test_ucs_nexp(self):
        path, cost, nexp = uniform_cost('chi','new',map_distances,return_cost=True,return_nexp=True)
        self.assertEqual(nexp, 11)
    def test_sld_providence(self):
        sld = heuristic_sld_providence('chi')
        self.assertEqual(sld, 833)
    def test_astar_path(self):
        path = astar_search('pit','pro', map_distances, heuristic_sld_providence, return_cost=False, return_nexp=False)
        self.assertEqual(path, ['pit', 'phi', 'new', 'pro'])
    def test_astar_cost(self):
        path, cost = astar_search('pit','pro', map_distances, heuristic_sld_providence, return_cost=True, return_nexp=False)
        self.assertEqual(cost, 583)
    def test_astar_nexp(self):
        path, cost, nexp = astar_search('pit','pro', map_distances, heuristic_sld_providence, return_cost=True, return_nexp=True)
        self.assertEqual(nexp, 5)
        

class Tests_Problem2(unittest.TestCase):
    def test_manhattan(self):
        state_test = np.array([[0,4,2],[1,3,5],[6,7,8]])
        goal = np.array([[0,1,2],[3,4,5],[6,7,8]])
        self.assertEqual(heuristic_manhattan(state_test, goal), 4)
    def test_misplaced(self):
        state_test = np.array([[0,4,2],[1,3,5],[6,7,8]])
        goal = np.array([[0,1,2],[3,4,5],[6,7,8]])
        self.assertEqual(heuristic_misplaced(state_test, goal), 3)
    def test_parity_true(self):
        state_test = np.array([[0,4,2],[1,3,5],[6,7,8]])
        goal = np.array([[0,1,2],[3,4,5],[6,7,8]])
        self.assertTrue(check_parity(state_test, goal))
    def test_parity_false(self):
        state_test = np.array([[0,2,1],[3,4,5],[6,7,8]])
        goal = np.array([[0,1,2],[3,4,5],[6,7,8]])
        self.assertFalse(check_parity(state_test, goal))
    def test_adjacent(self):
        state_test = np.array([[1,2,5],[3,0,4],[6,7,8]])
        adjacent = adjacent_states(state_test)
        adjacent_set = set()
        for k in range(len(adjacent)):
            adjacent_set.add(tuple(map(tuple,adjacent[k])))    
        answer = {((1, 0, 5), (3, 2, 4), (6, 7, 8)),
                  ((1, 2, 5), (0, 3, 4), (6, 7, 8)),
                  ((1, 2, 5), (3, 4, 0), (6, 7, 8)),
                  ((1, 2, 5), (3, 7, 4), (6, 0, 8))}
        self.assertEqual(adjacent_set, answer)
    def test_astar_8tile(self):
        goal = np.array([[0,1,2],[3,4,5],[6,7,8]])
        state_test = np.array([[1,4,2],[3,5,8],[6,7,0]])
        moves, cost, nexp = astar_8tile(state_test, goal,state_graph=adjacent_states, heuristic=heuristic_manhattan, return_cost=True, return_nexp=True)
        # make sure the path is in tuples
        moves_tuple = []
        for k in range(len(moves)):
            moves_tuple.append(tuple(map(tuple, moves[k])))    
        self.assertEqual(moves_tuple, [((1, 4, 2), (3, 5, 8), (6, 7, 0)), 
                                       ((1, 4, 2), (3, 5, 0), (6, 7, 8)), 
                                       ((1, 4, 2), (3, 0, 5), (6, 7, 8)), 
                                       ((1, 0, 2), (3, 4, 5), (6, 7, 8)), 
                                       ((0, 1, 2), (3, 4, 5), (6, 7, 8))])
        self.assertEqual(cost, 4)
        self.assertEqual(nexp, 5)
    def test_generate1(self):
        goal = np.array([[0,1,2],[3,4,5],[6,7,8]])
        n_moves=15
        puzzle, steps = generate_puzzle(goal, n_moves)
        solution, cost, nexp = astar_search(puzzle, goal, heuristic_manhattan, return_cost=True, return_nexp=True)
        self.assertTrue(cost<=n_moves)
        self.assertTrue(len(solution)<=(n_moves+1) and len(solution)>0)
    def test_generate2(self):
        goal = np.array([[0,1,2],[3,4,5],[6,7,8]])
        puzzle, steps = generate_puzzle(goal, n_moves=0)
        self.assertTrue((puzzle==goal).all())


[Back to top](#top)